In [1]:
import os, sys
from pathlib import Path

CURRENT_DIR = Path(os.getcwd())
PJROOT_DIR = CURRENT_DIR.parent.parent
LIB_PACKAGE_MAN_DIR = PJROOT_DIR / "airflow"

sys.path.append(str(LIB_PACKAGE_MAN_DIR))

In [2]:
import yaml, json

In [3]:
from lib_package_man import PackageManBackup, MakePlaybook

In [12]:
self = MakePlaybook()

In [13]:
self.df.index

Index(['192.168.0.100', '192.168.0.101', '192.168.0.134', '192.168.0.137',
       '192.168.0.145', '192.168.0.135'],
      dtype='object', name='ssh_ip_address')

In [14]:
self.df

,ssh_auth_method,ssh_user,ssh_password,ssh_secretkey_path,name,package_manager,env
ssh_ip_address,,,,,,,
192.168.0.100,password,$ANSIBLE_USER,$ANSIBLE_SSH_PASS,$ANSIBLE_SSH_PRIVATE_KEY_FILE,home_gw,apt,dev
192.168.0.101,password,$ANSIBLE_USER,$ANSIBLE_SSH_PASS,NaN,stdb1,apt,dev
192.168.0.134,password,$ANSIBLE_USER,$ANSIBLE_SSH_PASS,NaN,grafana1,apt,dev
192.168.0.137,password,$ANSIBLE_USER,$ANSIBLE_SSH_PASS,NaN,influxdb2,apt,dev
192.168.0.145,password,$ANSIBLE_USER,$ANSIBLE_SSH_PASS,$ANSIBLE_SSH_PRIVATE_KEY_FILE,django05,apt,dev
192.168.0.135,password,$ANSIBLE_USER,$ANSIBLE_SSH_PASS,$ANSIBLE_SSH_PRIVATE_KEY_FILE,pacmng1,apt,dev


In [15]:
package = "python3-pip"
post_upgrade_task = ""
service_name = ""
package_manager = "apt"
env = "dev"
# ssh_ip_address = "192.168.0.135"
self.make_upgrade_package_inventory_and_playbook(
    package, 
    post_upgrade_task=post_upgrade_task, 
    service_name=service_name, 
    package_manager=package_manager,
    env=env,
    # ssh_ip_address=ssh_ip_address,
)

Call MakePlaybook.make_get_packages_playbook method
Playbook /home/ogs-digilife/package_man/playbook/upgrade_package_playbook/upgrade-package.yml was made
Call MakePlaybook.make_inventory method
Inventory file /home/ogs-digilife/package_man/playbook/upgrade_package_playbook/inventory.yml was made



In [9]:
kwargs = {
    "package_manager": "apt",
    "env": "dev",
    "ssh_ip_address": "192.168.0.100"
}
package = "openssh-server"
post_upgrade_task = "reloaded"
service_name = "sshd"

# def make_package_upgrade_playbook(self, package, post_upgrade_task="", service_name="", **kwargs):
stdout = "Call MakePlaybook.make_get_packages_playbook method"
print(stdout)

# **kwargsでupgrade対象のhostをfilterする。
self.filter_hosts(**kwargs)

print(self.df.columns.tolist())

playbook = []
for ssh_ip_address in self.df.index:
    hostname = self.df.loc[ssh_ip_address, "name"]
    manager = self.df.loc[ssh_ip_address, "package_manager"]
    play = self._make_package_upgrade_play(ssh_ip_address, hostname, manager, package, post_upgrade_task, service_name)
    playbook.append(play)

print()
print(yaml.dump(playbook, sort_keys=False))

Call MakePlaybook.make_get_packages_playbook method
['ssh_auth_method', 'ssh_user', 'ssh_password', 'ssh_secretkey_path', 'name', 'package_manager', 'env']

- name: Upgrade home_gw_192.168.0.100's openssh-server package to latest play
  hosts: home_gw_192.168.0.100
  become: 'yes'
  tasks:
  - name: Update host home_gw_192.168.0.100's apt repositories cache task
    apt:
      update_cache: 'yes'
      cache_valid_time: 3600
  - name: Upgrade home_gw_192.168.0.100's openssh-server package to latest task
    apt:
      name: openssh-server
      state: latest
    notify: reloaded sshd
  handlers:
  - name: reloaded sshd
    service:
      name: sshd
      state: reloaded



In [17]:
kwargs = {
    "package_manager": "apt",
    "env": "dev",
    "name": "grafana1"
}
# 
# def filter_hosts(self, **kwargs):
filter_keys = kwargs.keys()

# filterが設定されていない場合は、何もせずreturnする
if len(filter_keys)==0:
    # return
    print("no filter data")

# validation & filter
correct_keys = ["ssh_ip_address"] + self.df.columns.tolist()
for k in filter_keys:
    # 正しいfilterが設定されてなければexit
    if not k in correct_keys:
        stdout = f'Exception: incorrect key "{k}" was set to filter --> exit process'
        sys.exit(stdout)

    fv = kwargs[k]
    self.df = self.df[self.df[k]==fv]

# validation2 -> レコード数が0になってしまった場合は、例外を発生させる。
if self.df.shape[0] == 0:
        stdout = f'Exception: No upgrade target host!! --> exit process'
        sys.exit(stdout)

self.df
        

,ssh_auth_method,ssh_user,ssh_password,ssh_secretkey_path,name,package_manager,env
ssh_ip_address,,,,,,,
192.168.0.134,password,$ANSIBLE_USER,$ANSIBLE_SSH_PASS,NaN,grafana1,apt,dev


In [7]:
kwargs = {
    "package_manager": "apt"
}
# def make_package_upgrade_playbook(self, package, post_pgrade_task="", service_name="", **kwargs):
stdout = "Call make_package_upgrade_playbook method"
print(stdout)

# 対象をfilter

    

hosts

Call MakePlaybook.make_get_packages_playbook method


['home_gw', 'stdb1', 'grafana1', 'influxdb2', 'django05']

In [6]:
ssh_ip_address = "192.168.0.100"
hostname = "home_gw"
manager = "apt"
package = "python-pip"
post_upgrade_task = "reboot"
service_name = "pip"

play = self._make_package_upgrade_play(ssh_ip_address, hostname, manager, package, post_upgrade_task, service_name)
print(yaml.dump(play, sort_keys=False))

name: Upgrade home_gw_192.168.0.100's python-pip package to latest play
hosts: home_gw_192.168.0.100
become: 'yes'
tasks:
- name: Update host home_gw_192.168.0.100's apt repositories cache task
  apt:
    update_cache: 'yes'
    cache_valid_time: 3600
- name: Upgrade home_gw_192.168.0.100's python-pip package to latest task
  apt:
    name: python-pip
    state: latest
  notify: REBOOT home_gw_192.168.0.100
handlers:
- name: REBOOT home_gw_192.168.0.100
  reboot:
    pre_reboot_delay: 10
    post_reboot_delay: 30
    test_command: uptime
    reboot_timeout: 300



In [14]:
handlers_name = "reloaded python3-pip"
handlers_service_name = "sshd"
handlers_service_state="reloaded"

# def _make_handlers_for_package_upgrade_play(self, handlers_name, handlers_service_name, handlers_service_state):
handlers = []

handler = {}
handler["name"] = handlers_name
handler["service"] = {
    "name": handlers_service_name,
    "state": handlers_service_state
}

handlers.append(handler)

print(yaml.dump(handlers, sort_keys=False))

- name: reloaded python3-pip
  service:
    name: sshd
    state: reloaded



In [11]:
package_manager = "apt"

ssh_ip_address = "192.168.0.100"
hostname = "home_gw"
manager = "apt"
package = "python3-pip"
post_upgrade_task="reloaded"

# パッケージの最新版への更新
# def make_package_upgrade_playbook(self):のサブメソッド
# def _make_package_upgrade_plays(package_manager, package, post_upgrade_task):
# dfをhgでfilter
pdf = self.df[self.df["package_manager"]==package_manager]

# playはhostごとに作成する
ssh_ip_addresses = pdf.index
print(pdf.columns)
package_plays = []
for ip in ssh_ip_addresses:
    package_plays.append(self._make_packageupdate_play(ip, hostname, package_manager, package, post_upgrade_task))

print(yaml.dump(package_plays, sort_keys=False))

Index(['ssh_auth_method', 'ssh_user', 'ssh_password', 'ssh_secretkey_path',
       'name', 'package_manager', 'env'],
      dtype='object')
- name: Upgrade home_gw_192.168.0.100's python3-pip package to latest play
  hosts: home_gw_192.168.0.100
  tasks:
  - name: Update host home_gw_192.168.0.100's apt repositories cache task
    apt:
      update_cache: 'yes'
      cache_valid_time: 3600
  - name: Upgrade home_gw_192.168.0.100's python3-pip package to latest task
    apt:
      name: python3-pip
      state: latest
    notify: reloaded python3-pip
- name: Upgrade stdb1_192.168.0.101's python3-pip package to latest play
  hosts: stdb1_192.168.0.101
  tasks:
  - name: Update host stdb1_192.168.0.101's apt repositories cache task
    apt:
      update_cache: 'yes'
      cache_valid_time: 3600
  - name: Upgrade stdb1_192.168.0.101's python3-pip package to latest task
    apt:
      name: python3-pip
      state: latest
    notify: reloaded python3-pip
- name: Upgrade grafana1_192.168.0.1

In [5]:
# パッケージの最新版への更新
# def make_package_upgrade_playbook(self):
stdout = "Call MakePlaybook.make_get_packages_playbook method"
print(stdout)

playbook = []

hosts = self.df["package_manager"].drop_duplicates().tolist()

for hg in hosts:
    # OS情報収集結果を出力するためのフォルダを作成するplayを作成してplaybookに追加
    # playbook.append(self._make_get_os_version_plays(hg))
    playbook += self._make_get_package_plays(hg)

Call MakePlaybook.make_get_packages_playbook method


['apt']

In [ ]:
# パッケージの最新版への更新
# post_upgrade_task = ""|"reloaded"|"restarted"|"reboot"|
# def make_package_upgrade_playbook(self, package, post_upgrade_task=""):のサブメソッド
# def _make_package_upgrade_plays(self, package, package_manager, post_upgrade_task=""):のサブメソッド
# def _make_package_update_play(self, package, ssh_ip_address, post_upgrade_task=""):のサブメソッド
def _make_tasks_for_package_upgrade_play(self, hostname, manager, package, post_upgrade_task=""):
    tasks = []
    
    # repositories cach updateタスク
    dct = {}
    dct["name"] = f'Update host {hostname}\'s {manager} repositories cache'
    dct[manager] = {
        "update_cache": "yes",
        "cache_valid_time": 3600,
    }
    tasks.append(dct)
    
    # package更新タスク
    dct = {}
    dct["name"] = f'Upgrade {hostname}\'s {package} package to latest'
    dct[f'{manager}'] = {
        "name": package,
        "state": "latest"
    }
    # post_upgrade_taskが指定されていた場合は、notifyをセット。
    if post_upgrade_task == "reloaded" or post_upgrade_task == "restarted":
        dct["notify"] = f'{post_upgrade_task} {package}'
    elif post_upgrade_task == "reboot":
        dct["notify"] = "REBOOT SYSTEM"
    elif post_upgrade_task == "":
        pass
    else:
        stdout = f'WARNING!!!: unknown post_upgrade_task "{post_upgrade_task}" --> ignored\n'
        print(stdout)
    tasks.append(dct)
    
    return tasks